<a href="https://colab.research.google.com/github/EasonJI0403/Independent_Study_data/blob/main/playerScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import json
import pandas as pd
from collections import defaultdict

# 標準化數據
def normalize_stat(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val) * 100

# 計算分數
def calculate_score(normalized_value):
    return int(round(normalized_value / 100 * 99)) + 1

# 球員數據
with open('/content/drive/MyDrive/P_Json/P_Players_performance_23_24.json', encoding='utf-8') as f:
    data = json.load(f)

player_data = pd.DataFrame(data)

# 定義統計數據範圍（最小值，最大值），基於聯盟平均水平和頂級表現者
stat_ranges = {
    "points": (0, 27),
    "field_goals_two_made": (0, 9),
    "field_goals_two_pct": (0, 60),
    "field_goals_three_made": (0, 4),
    "field_goals_three_pct": (0, 40),
    "free_throws_made": (0, 7),
    "free_throws_pct": (0, 90),
    "rebounds": (0, 13),
    "assists": (0, 7),
    "steals": (0, 5),
    "blocks": (0, 3),
    "turnovers": (5, 0),  # 反向比例
    "fouls": (5, 0)  # 反向比例
}

# 確保所有需要的統計數據都是數字類型
for stat in stat_ranges.keys():
    if stat in player_data:
        player_data[stat] = pd.to_numeric(player_data[stat], errors='coerce')

# 計算標準化統計數據
normalized_stats = {}
for stat, (min_val, max_val) in stat_ranges.items():
    if stat in player_data:
        normalized_stats[stat] = player_data[stat].apply(lambda x: normalize_stat(x, min_val, max_val) if pd.notnull(x) else np.nan)

# 定義類別權重
category_weights = {
    "得分能力": {
        "field_goals_two_made": 0.3,
        "field_goals_two_pct": 0.1,
        "field_goals_three_made": 0.3,
        "field_goals_three_pct": 0.1,
        "free_throws_made": 0.1,
        "free_throws_pct": 0.1
    },
    "外線得分": {"field_goals_three_made": 0.7, "field_goals_three_pct": 0.3},
    "組織能力": {"assists": 0.7, "turnovers": 0.3},
    "防守能力": {"steals": 0.8, "blocks": 0.8, "fouls": 0.4},
    "籃板能力": {"rebounds": 1.0},
    "運動能力": {"points": 0.3, "rebounds": 0.3, "blocks": 0.4},
}

# 定義新的類別權重
category_overall_weights = {
    "得分能力": 0.3,
    "外線得分": 0.2,
    "組織能力": 0.1,
    "防守能力": 0.2,
    "籃板能力": 0.1,
    "運動能力": 0.1
}

# 修改計算總體分數的函數
def calculate_weighted_overall_score(scores):
    weighted_sum = sum(scores[category] * weight for category, weight in category_overall_weights.items())
    return calculate_score(weighted_sum)

# 存儲所有球員的結果
teams = defaultdict(list)
# 遍歷每個球員
for idx in range(len(player_data)):
    player_result = {
        "player": player_data['player'].iloc[idx],
        "team": player_data['team'].iloc[idx],
        "scores": {},
    }

    # 計算類別分數
    for category, weights in category_weights.items():
        score = 0
        for stat, weight in weights.items():
            if stat in normalized_stats:
                stat_value = normalized_stats[stat].iloc[idx]
                score += stat_value * weight

        category_score = calculate_score(score)
        player_result["scores"][category] = category_score

    # 計算加權總體分數
    overall_score = calculate_weighted_overall_score(player_result["scores"])
    player_result["overall_score"] = overall_score

    # 將球員添加到對應的隊伍
    teams[player_result["team"]].append(player_result)

# 創建最終結果列表
final_results = []

# 將每個隊伍的球員添加到最終結果
for team, players in teams.items():
    # 對每個隊伍的球員按總體分數降序排序
    players_sorted = sorted(players, key=lambda x: x['overall_score'], reverse=True)
    final_results.append({
        "team": team,
        "players": players_sorted
    })

# 將結果轉換成 JSON 並輸出到文件
with open('playerScoresT.json', 'w', encoding='utf-8') as f:
    json.dump(final_results, f, ensure_ascii=False, indent=4)

print("已成功將結果輸出到 playerScoresT.json")

已成功將結果輸出到 playerScoresT.json
